In [222]:
import pandas as pd
import io

In [223]:
def analyze_train_data(split_transfers: bool):
    """
    Analyzes the train data to generate two tables based on the 'split_transfers' parameter.

    This function now reads the "Trenes.csv" file directly from disk.
    Please ensure the file is in the same directory as this script.

    Args:
        split_transfers (bool):
            - If True, a trip with transfers is split into multiple segments.
            - If False, a trip with transfers is considered a single journey from start to end.

    Returns:
        tuple: A tuple containing two pandas DataFrames (routes_df, stations_df).
    """
    # Read the CSV file directly from disk, now with comma as separator.
    try:
        df = pd.read_csv("Trenes.csv", sep=',')
    except FileNotFoundError:
        print("Error: 'Trenes.csv' file not found. Please make sure the file is in the same directory.")
        return None, None

    # Removed the initial filter for 'Cerc' trains to handle them on a segment-by-segment basis.

    # --- Step 1: Process and normalize the data based on 'split_transfers' ---
    processed_trips = []
    station_counts = {}

    for _, row in df.iterrows():
        origin = row['Origen']
        destination = row['Destino']
        transfers_str = str(row['Transbordos'])
        transfer_stations = transfers_str.split(',') if transfers_str and transfers_str != 'nan' else []
        train_types = row['Tipo'].split(',')
        
        # Ensure the number of transfer stations and train types match
        if len(transfer_stations) + 1 != len(train_types):
            print(f"Warning: Mismatch in data for row with origin '{origin}'. Skipping.")
            continue

        if split_transfers:
            # Mode 1: Split a trip with transfers into multiple segments
            current_origin = origin
            for i, transfer_station in enumerate(transfer_stations):
                current_destination = transfer_station.strip()
                # Only process this segment if it's not a 'Cerc' train
                if train_types[i].strip() != 'Cerc':
                    processed_trips.append({
                        'Origen': current_origin,
                        'Destino': current_destination,
                        'Tipo': train_types[i].strip()
                    })
                current_origin = current_destination
            
            # Add the final segment from the last transfer to the destination, if it's not 'Cerc'
            if train_types[-1].strip() != 'Cerc':
                processed_trips.append({
                    'Origen': current_origin,
                    'Destino': destination,
                    'Tipo': train_types[-1].strip()
                })

            # Count all stations (origin, transfers, destination)
            station_counts[origin] = station_counts.get(origin, 0) + 1
            for station in transfer_stations:
                station_counts[station.strip()] = station_counts.get(station.strip(), 0) + 1
            station_counts[destination] = station_counts.get(destination, 0) + 1
            
        else:
            # Mode 2: Consider the whole trip as a single journey
            # Build the 'Tipo' string for the trip, explicitly excluding 'Cerc' trains
            filtered_types = [t.strip() for t in train_types if t.strip() in ['AV', 'Reg', 'Intercity']]
            
            if filtered_types: # Only add the trip if there are non-Cerc trains
                processed_trips.append({
                    'Origen': origin,
                    'Destino': destination,
                    'Tipo': ','.join(filtered_types)
                })

                # Count only origin and destination stations
                station_counts[origin] = station_counts.get(origin, 0) + 1
                station_counts[destination] = station_counts.get(destination, 0) + 1
    
    # Create a DataFrame from the processed trips for easier analysis
    trips_df = pd.DataFrame(processed_trips)

    # --- Step 2: Create Table 1 (Train Routes) ---
    routes_data = {}
    for _, trip in trips_df.iterrows():
        route = (trip['Origen'], trip['Destino'])
        if route not in routes_data:
            routes_data[route] = {'AV': 0, 'Reg': 0} # 'Reg' will accumulate 'Reg' and 'Intercity'
        
        types_in_trip = trip['Tipo'].split(',')
        for t in types_in_trip:
            if t.strip() == 'AV':
                routes_data[route]['AV'] += 1
            elif t.strip() in ['Reg', 'MD', 'Intercity']: # Include 'Intercity' here for 'times not AV'
                routes_data[route]['Reg'] += 1

    routes_df = pd.DataFrame.from_dict(routes_data, orient='index')
    routes_df.index.names = ['Departure station', 'Arrival station']
    routes_df = routes_df.reset_index()

    # Normalize routes (A,B) and (B,A) to be the same and rename columns consistently
    normalized_routes_data = {}
    for index, row in routes_df.iterrows():
        station1 = row['Departure station']
        station2 = row['Arrival station']
        
        # Create a canonical key for the route (e.g., ('A', 'B') for both A->B and B->A)
        canonical_key = tuple(sorted((station1, station2)))
        
        if canonical_key not in normalized_routes_data:
            normalized_routes_data[canonical_key] = {'AV': 0, 'Reg': 0}
        
        # Use the original column names before renaming
        normalized_routes_data[canonical_key]['AV'] += row['AV']
        normalized_routes_data[canonical_key]['Reg'] += row['Reg']

    # Recreate routes_df from normalized data
    routes_df = pd.DataFrame.from_dict(normalized_routes_data, orient='index')
    routes_df.index.names = ['Station1', 'Station2'] # Temporary names
    routes_df = routes_df.reset_index()
    
    # Rename columns as requested
    routes_df.columns = [
        'Departure station', 
        'Arrival station',   
        'times AV',
        'times not AV'
    ]
    
    # Sort by total times used
    routes_df['Total'] = routes_df['times AV'] + routes_df['times not AV']
    routes_df = routes_df.sort_values(by='Total', ascending=False).drop(columns='Total')

    # The rest of the function remains the same for station_df
    # --- Step 3: Create Table 2 (Station Usage) ---
    # Automatically get coordinates using the geopy library and caching.
    unique_stations = sorted(list(station_counts.keys()))
    station_coordinates = get_station_coordinates(unique_stations)

    stations_df = pd.DataFrame(
        list(station_counts.items()), 
        columns=['name of station', 'number of times used']
    )
    
    # Add new columns for latitude and longitude
    stations_df['latitude'] = stations_df['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[0]
    )
    stations_df['longitude'] = stations_df['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[1]
    )

    stations_df = stations_df.sort_values(by='number of times used', ascending=False)
    
    return routes_df, stations_df

In [224]:
    print("--- Analysis Mode 1: Considering trips as a single journey (no splits) ---")
    routes_table_no_split, stations_table_no_split = analyze_train_data(split_transfers=False)
    print("\nTable of Train Routes:")
    print(routes_table_no_split.to_string(index=False))
    print("\nTable of Station Usage:")
    print(stations_table_no_split.to_string(index=False))

    print("\n\n" + "="*80)
    
    print("\n--- Analysis Mode 2: Splitting trips with transfers into segments ---")
    routes_table_split, stations_table_split = analyze_train_data(split_transfers=True)
    print("\nTable of Train Routes:")
    print(routes_table_split.to_string(index=False))
    print("\nTable of Station Usage:")
    print(stations_table_split.to_string(index=False))

--- Analysis Mode 1: Considering trips as a single journey (no splits) ---

Table of Train Routes:
          Departure station             Arrival station  times AV  times not AV
            Madrid - Atocha         Zaragoza - Delicias        40             0
          Barcelona - Sants         Zaragoza - Delicias        15             0
                       Jaca         Zaragoza - Delicias         0             5
          Barcelona - Sants               Valence - TGV         4             0
                    Granada             Madrid - Atocha         3             0
                     Bedous                      Oloron         0             3
          Barcelona - Sants                  Strasbourg         2             0
                     Málaga         Zaragoza - Delicias         2             0
                    Granada         Zaragoza - Delicias         2             0
                 Düsseldorf                  Strasbourg         2             0
                   Ar

In [225]:
stations_table_split

,name of station,number of times used,latitude,longitude
0,Zaragoza - Delicias,70,41.658573,-0.911388
2,Madrid - Atocha,46,40.406975,-3.688601
14,Barcelona - Sants,24,41.379007,2.140004
31,Valence - TGV,8,44.991932,4.978434
35,Jaca,7,42.577732,-0.538463
4,Bordeaux,5,44.825523,-0.555650
27,Gent - St Pieters,4,51.035524,3.710716
37,Oloron,4,43.182743,-0.608239
36,Bedous,4,42.995221,-0.603744
33,Chambéry,4,45.588369,5.889405


In [226]:
unique_stations_split = list(stations_table_split["name of station"])
unique_stations_no_split = list(stations_table_no_split["name of station"])

In [227]:
import time
import os
import csv
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.extra.rate_limiter import RateLimiter

def get_station_coordinates(station_names: list) -> dict:
    """
    Uses the Nominatim geocoding service to find the latitude and longitude
    for a list of station names, with caching to a CSV file.

    Args:
        station_names (list): A list of strings, where each string is a station name.

    Returns:
        dict: A dictionary with station names as keys and (latitude, longitude)
              tuples as values.
    """
    COORD_FILE = "station_coordinates.csv"
    station_coords = {}

    # 1. Load existing coordinates from CSV if file exists
    if os.path.exists(COORD_FILE):
        with open(COORD_FILE, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Skip header if it exists
            header = next(reader, None)
            if header and header[0] != 'station_name': # Simple check for header
                file.seek(0) # Rewind if no header
            
            for row in reader:
                if len(row) == 3:
                    name, lat_str, lon_str = row
                    try:
                        # Convert to float, handle 'None' strings
                        latitude = float(lat_str) if lat_str != 'None' else None
                        longitude = float(lon_str) if lon_str != 'None' else None
                        station_coords[name] = (latitude, longitude)
                    except ValueError:
                        # Handle cases where conversion fails (e.g., malformed data)
                        station_coords[name] = (None, None)
                else:
                    print(f"Warning: Skipping malformed row in {COORD_FILE}: {row}")

    # Initialize geolocator and RateLimiter
    geolocator = Nominatim(user_agent="train_station_geocoder")
    # Use a delay of 2 seconds as requested, to respect API limits
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

    # 2. Query for missing coordinates
    for s in station_names:
        if s not in station_coords or station_coords[s] == (None, None):
            # Only query if the station is not in our cache or if its coordinates are None
            q = f"{s} station" # Added "Spain" for better accuracy
            try:
                loc = geocode(q, exactly_one=True)
                if loc:
                    station_coords[s] = (loc.latitude, loc.longitude)
                else:
                    station_coords[s] = (None, None)
            except (GeocoderTimedOut, GeocoderServiceError) as e:
                print(f"Error geocoding '{s}': {e}")
                station_coords[s] = (None, None)
        
    # 3. Save all current coordinates back to CSV
    with open(COORD_FILE, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['station_name', 'latitude', 'longitude']) # Write header
        for name, coords in station_coords.items():
            writer.writerow([name, coords[0], coords[1]])
            
    return station_coords

In [228]:
station_coordinates = get_station_coordinates(unique_stations_no_split)

In [229]:
    stations_table_split['latitude'] = stations_table_split['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[0]
    )
    stations_table_split['longitude'] = stations_table_split['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[1]
    )

    stations_table_no_split['latitude'] = stations_table_no_split['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[0]
    )
    stations_table_no_split['longitude'] = stations_table_no_split['name of station'].apply(
        lambda name: station_coordinates.get(name, (None, None))[1]
    )

In [230]:
stations_table_split

,name of station,number of times used,latitude,longitude
0,Zaragoza - Delicias,70,41.658573,-0.911388
2,Madrid - Atocha,46,40.406975,-3.688601
14,Barcelona - Sants,24,41.379007,2.140004
31,Valence - TGV,8,44.991932,4.978434
35,Jaca,7,42.577732,-0.538463
4,Bordeaux,5,44.825523,-0.555650
27,Gent - St Pieters,4,51.035524,3.710716
37,Oloron,4,43.182743,-0.608239
36,Bedous,4,42.995221,-0.603744
33,Chambéry,4,45.588369,5.889405


In [231]:
import folium

def get_interpolated_color(av_percentage):
    """
    Interpolates a color between blue (0% AV) and red (100% AV).
    """
    # Red: (255, 0, 0)
    # Blue: (0, 0, 255)
    
    # Ensure percentage is between 0 and 1
    av_percentage = max(0, min(1, av_percentage))

    # Interpolate RGB components
    red_component = int(255 * av_percentage)
    blue_component = int(255 * (1 - av_percentage))
    green_component = 0 # No green in this interpolation

    # Convert to hex color string
    return f'#{red_component:02x}{green_component:02x}{blue_component:02x}'
    
def create_interactive_map(stations_df: pd.DataFrame, routes_df: pd.DataFrame, output_filename="train_stations_map.html", min_radius=5, max_radius=30, min_line_width=2, max_line_width=10):
    """
    Creates an interactive HTML map with station points and route lines, and saves it to a file.
    The size of the points is proportional to the square root of 'number of times used',
    scaled to fit between min_radius and max_radius.
    The width of the lines is proportional to the square root of 'total times used',
    scaled to fit between min_line_width and max_line_width.
    The color of the lines interpolates between blue (0% AV) and red (100% AV).

    Args:
        stations_df (pd.DataFrame): DataFrame containing station name, number of times used,
                                    latitude, and longitude.
        routes_df (pd.DataFrame): DataFrame containing route information (Departure, Arrival, times AV, times not AV).
        output_filename (str): The name of the HTML file to save the map to.
        min_radius (int): The minimum radius for the circle markers.
        max_radius (int): The maximum radius for the circle markers.
        min_line_width (int): The minimum width for the route lines.
        max_line_width (int): The maximum width for the route lines.
    """
    # Filter out stations without coordinates
    stations_df = stations_df.dropna(subset=['latitude', 'longitude'])

    if stations_df.empty:
        print("No stations with valid coordinates to display on the map.")
        return

    # Create a dictionary for quick station coordinate lookup
    station_coords_lookup = stations_df.set_index('name of station')[['latitude', 'longitude']].apply(tuple, axis=1).to_dict()

    # Center the map on Europe and set an appropriate initial zoom level
    m = folium.Map(location=[50, 10], zoom_start=4)

    # --- Add Markers for each station ---
    # Calculate raw square root values for radius
    raw_radii = stations_df['number of times used'].apply(lambda x: x**0.5)

    # Find min and max of these raw radii
    min_raw_r = raw_radii.min()
    max_raw_r = raw_radii.max()

    # Handle case where all usage counts are the same (to avoid division by zero)
    if max_raw_r == min_raw_r:
        scaled_radii = pd.Series([ (min_radius + max_radius) / 2 ] * len(stations_df), index=stations_df.index)
    else:
        # Normalize raw radii to the [min_radius, max_radius] range
        scaled_radii = min_radius + (raw_radii - min_raw_r) * (max_radius - min_radius) / (max_raw_r - min_raw_r)
    
    stations_df['scaled_radius'] = scaled_radii

    for idx, row in stations_df.iterrows():
        station_name = row['name of station']
        times_used = row['number of times used']
        lat = row['latitude']
        lon = row['longitude']
        radius = row['scaled_radius'] # Use the newly computed scaled radius

        folium.CircleMarker(
            location=[lat, lon],
            radius=radius,
            color='lightblue',       # Set border color to white
            weight=2,            # Border thickness
            fill=True,
            fill_color='darkblue', # Fill color for the circle
            fill_opacity=0.7,
            popup=f"<b>{station_name}</b><br>{times_used} times"
        ).add_to(m)

    # --- Add Lines for each route ---
    # Calculate raw square root values for line width
    # Ensure 'Total' column exists and is numeric
    if 'Total' not in routes_df.columns:
        routes_df['Total'] = routes_df['times AV'] + routes_df['times not AV']

    raw_widths = routes_df['Total'].apply(lambda x: x**0.5)
    min_raw_w = raw_widths.min()
    max_raw_w = raw_widths.max()

    # Handle case where all total times are the same
    if max_raw_w == min_raw_w:
        scaled_widths = pd.Series([ (min_line_width + max_line_width) / 2 ] * len(routes_df), index=routes_df.index)
    else:
        scaled_widths = min_line_width + (raw_widths - min_raw_w) * (max_line_width - min_line_width) / (max_raw_w - min_raw_w)
    
    routes_df['scaled_width'] = scaled_widths

    for idx, row in routes_df.iterrows():
        dep_station = row['Departure station']
        arr_station = row['Arrival station']
        times_av = row['times AV']
        times_not_av = row['times not AV']
        total_times = row['Total']
        line_width = row['scaled_width']

        dep_coords = station_coords_lookup.get(dep_station)
        arr_coords = station_coords_lookup.get(arr_station)

        if dep_coords and arr_coords:
            # Calculate AV percentage for color interpolation
            av_percentage = times_av / total_times if total_times > 0 else 0
            line_color = get_interpolated_color(av_percentage)

            folium.PolyLine(
                locations=[dep_coords, arr_coords],
                color=line_color,
                weight=line_width,
                opacity=0.8,
                popup=f"<b>{dep_station} - {arr_station}</b><br>"
                      f"Total: {total_times}<br>"
                      f"AV: {times_av}<br>"
                      f"Non-AV: {times_not_av}"
            ).add_to(m)

    # Save the map to an HTML file
    m.save(output_filename)
    print(f"Interactive map saved to {output_filename}")

In [232]:
# Generate and save the interactive map
print("\nGenerating and saving interactive map...")
create_interactive_map(stations_table_split, routes_table_split, output_filename="train_stations_map_split.html", min_radius=3, max_radius=30, min_line_width=2, max_line_width=15)
create_interactive_map(stations_table_no_split, routes_table_no_split, output_filename="train_stations_map.html", min_radius=3, max_radius=30, min_line_width=2, max_line_width=15)

print("\nMap generation complete! You can now open 'train_stations_map.html' in your web browser to view the interactive map.")


Generating and saving interactive map...
Interactive map saved to train_stations_map_split.html
Interactive map saved to train_stations_map.html

Map generation complete! You can now open 'train_stations_map.html' in your web browser to view the interactive map.


In [196]:
stations_table_split

,name of station,number of times used,latitude,longitude
0,Zaragoza - Delicias,67,41.658573,-0.911388
2,Madrid - Atocha,43,40.406975,-3.688601
14,Barcelona - Sants,24,41.379007,2.140004
31,Valence - TGV,8,44.991932,4.978434
35,Jaca,7,42.577732,-0.538463
4,Bordeaux,5,44.825523,-0.555650
9,Granada,4,37.184037,-3.608893
37,Oloron,4,43.182743,-0.608239
27,Gent - St Pieters,4,51.035524,3.710716
36,Bedous,4,42.995221,-0.603744


In [197]:
routes_table_split

,Departure station,Arrival station,number of times travelled with an AV train,number of trains traveled with a Reg train
3,Madrid - Atocha,Zaragoza - Delicias,23,0
2,Zaragoza - Delicias,Madrid - Atocha,17,0
18,Zaragoza - Delicias,Barcelona - Sants,8,0
19,Barcelona - Sants,Zaragoza - Delicias,7,0
50,Jaca,Zaragoza - Delicias,0,3
...,...,...,...,...
59,Salou,Barcelona - Sants,0,1
60,Camp Tarragona,Valencia - Joaquín Sorolla,1,0
61,Valencia - Joaquín Sorolla,Madrid - Chamartín,1,0
62,Villanueva de Gállego,Zaragoza - Goya,0,1
